## A.1. PRETRAINED WEIGHTS FOR PREDICTION


In [24]:
import os, time, math, sys, pathlib, gc
from typing import Tuple, List, Dict
import numpy as np
from PIL import Image
from tabulate import tabulate

import tensorflow as tf
from tensorflow import keras

# ==== 1) CẤU HÌNH DỮ LIỆU ====
VAL_DIR = r"E:\Pycharm\Advanced-Reading-on-Computer-Vision\Images\CNN_MultiClass_data\validation"
# Thư mục val được kỳ vọng có cấu trúc:
# val/
#   cats/
#   dogs/
#   pandas/
TARGET_CLASSES = ["cats", "dogs", "panda"]
CLASS_TO_IDX = {c: i for i, c in enumerate(TARGET_CLASSES)}

In [25]:


# ==== 2) KHAI BÁO MODEL PRETRAINED (A.1) ====
# Theo tài liệu: dùng các model Keras Applications weights='imagenet' (không train) :contentReference[oaicite:3]{index=3}
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input as vgg19_pre
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input as vgg16_pre
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input as res50_pre, decode_predictions as resnet_decode
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input as incv3_pre, decode_predictions as inception_decode
from tensorflow.keras.applications.xception import Xception, preprocess_input as xcep_pre, decode_predictions as xception_decode
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input as mnv2_pre, decode_predictions as mobilenet_decode
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input as effb0_pre, decode_predictions as efficientnet_decode

# Mỗi model có size đầu vào & hàm decode riêng (Inception/Xception: 299; phần còn lại: 224) :contentReference[oaicite:4]{index=4}
MODEL_SPECS = {
    "VGG16":      {"builder": lambda: VGG16(weights="imagenet"),      "size": (224, 224), "pre": vgg16_pre,  "decode": resnet_decode},
    "VGG19":      {"builder": lambda: VGG19(weights="imagenet"),      "size": (224, 224), "pre": vgg19_pre,  "decode": resnet_decode},
    "ResNet50":   {"builder": lambda: ResNet50(weights="imagenet"),   "size": (224, 224), "pre": res50_pre,  "decode": resnet_decode},
    "InceptionV3":{"builder": lambda: InceptionV3(weights="imagenet"),"size": (299, 299), "pre": incv3_pre,  "decode": inception_decode},
    "Xception":   {"builder": lambda: Xception(weights="imagenet"),   "size": (299, 299), "pre": xcep_pre,   "decode": xception_decode},
    "MobileNetV2":{"builder": lambda: MobileNetV2(weights="imagenet"),"size": (224, 224), "pre": mnv2_pre,   "decode": mobilenet_decode},
    "EfficientNetB0":{"builder": lambda: EfficientNetB0(weights="imagenet"),"size": (224, 224), "pre": effb0_pre, "decode": efficientnet_decode},
}

# ==== 3) ĐỌC DANH SÁCH ẢNH & NHÃN (val) ====
def list_images_with_labels(root_dir: str) -> List[Tuple[str, int]]:
    items = []
    for cls in TARGET_CLASSES:
        cls_dir = os.path.join(root_dir, cls)
        if not os.path.isdir(cls_dir):
            raise FileNotFoundError(f"Thiếu thư mục lớp: {cls_dir}")
        for fname in os.listdir(cls_dir):
            if fname.lower().endswith((".jpg", ".jpeg", ".png")):
                items.append((os.path.join(cls_dir, fname), CLASS_TO_IDX[cls]))
    if len(items) == 0:
        raise RuntimeError(f"Không thấy ảnh nào trong {root_dir}")
    return items

VAL_ITEMS = list_images_with_labels(VAL_DIR)
print(f"[INFO] Tổng ảnh Validation: {len(VAL_ITEMS)} | Lớp: {TARGET_CLASSES}")

# ==== 4) TIỆN ÍCH LOAD & TIỀN XỬ LÝ ẢNH ====
def load_image(path: str, target_size: Tuple[int, int]) -> np.ndarray:
    img = Image.open(path).convert("RGB").resize(target_size, Image.BILINEAR)
    arr = np.asarray(img, dtype=np.float32)
    return arr

# ==== 5) MAP KẾT QUẢ IMAGENET -> {cats, dogs, pandas} ====
# A.1 cho ví dụ decode_predictions (class, description, prob) dựa trên weights ImageNet :contentReference[oaicite:5]{index=5}
# Ta map theo mô tả nhãn:
def map_imagenet_to_3classes(decoded_topk: List[Tuple[str, str, float]]) -> int:
    # Ưu tiên "panda" rồi "dog" rồi "cat" trong top-k
    for (_, desc, _) in decoded_topk:
        d = desc.lower()
        if "panda" in d:
            return CLASS_TO_IDX["panda"]
    for (_, desc, _) in decoded_topk:
        d = desc.lower()
        if "dog" in d:
            return CLASS_TO_IDX["dogs"]
    for (_, desc, _) in decoded_topk:
        d = desc.lower()
        if "cat" in d:
            return CLASS_TO_IDX["cats"]
    # nếu không rơi vào 3 nhóm trên, coi như sai (trả về -1)
    return -1

# ==== 6) HÀM ĐÁNH GIÁ 1 MODEL ====
def evaluate_model(model_name: str,
                   builder, input_size, preprocess_fn, decoder,
                   items: List[Tuple[str, int]],
                   batch_size: int = 16,
                   top_k: int = 3) -> Dict:
    # 6.1 load model & đo thời gian
    t0 = time.perf_counter()
    model = builder()
    load_s = time.perf_counter() - t0

    # 6.2 suy luận theo batch & đo thời gian/ảnh
    N = len(items)
    correct = 0
    unknown = 0
    predict_times = []

    # Chuẩn bị batch
    def batch_iter(seq, bs):
        for i in range(0, len(seq), bs):
            yield seq[i:i+bs]

    for batch in batch_iter(items, batch_size):
        # Load ảnh
        imgs = np.stack([load_image(p, input_size) for (p, _) in batch], axis=0)
        y_true = np.array([y for (_, y) in batch], dtype=np.int32)

        # Tiền xử lý theo model
        x = preprocess_fn(imgs.copy())

        # Dự đoán & thời gian
        t1 = time.perf_counter()
        preds = model.predict(x, verbose=0)
        t2 = time.perf_counter()
        dt = (t2 - t1) / len(batch)  # giây/ảnh
        predict_times.extend([dt] * len(batch))

        # Decode & map
        decoded = decoder(preds, top=top_k)
        y_pred_3 = []
        for i in range(len(batch)):
            mapped = map_imagenet_to_3classes(decoded[i])
            y_pred_3.append(mapped)
        y_pred_3 = np.array(y_pred_3, dtype=np.int32)

        # Đếm đúng/sai
        mask_known = (y_pred_3 != -1)
        unknown += int(np.sum(~mask_known))
        correct += int(np.sum((y_pred_3 == y_true) & mask_known))

    acc = correct / N
    avg_pred_sec = float(np.mean(predict_times))
    images_per_sec = 1.0 / avg_pred_sec if avg_pred_sec > 0 else float("nan")

    # Giải phóng
    del model
    gc.collect()
    tf.keras.backend.clear_session()

    return {
        "Model": model_name,
        "Accuracy": acc,
        "Load_s": load_s,
        "Pred_ms_per_img": avg_pred_sec * 1000.0,
        "Images_per_s": images_per_sec,
        "Unknown_mapped": unknown,
        "Total": N,
    }

# ==== 7) CHẠY THỬ TẤT CẢ MODEL & IN BẢNG ====
RESULTS = []
ORDER = [
    "VGG16", "VGG19", "ResNet50", "InceptionV3", "Xception", "MobileNetV2", "EfficientNetB0"
]
for name in ORDER:
    spec = MODEL_SPECS[name]
    print(f"\n=== Running {name} ===")
    out = evaluate_model(
        model_name=name,
        builder=spec["builder"],
        input_size=spec["size"],
        preprocess_fn=spec["pre"],
        decoder=spec["decode"],
        items=VAL_ITEMS,
        batch_size=16,
        top_k=3
    )
    RESULTS.append(out)
    print(out)

# Code đã hoàn thành


[INFO] Tổng ảnh Validation: 3000 | Lớp: ['cats', 'dogs', 'panda']

=== Running VGG16 ===


KeyboardInterrupt: 